<a href="https://colab.research.google.com/github/Gus-1003/Projeto_PalmaS/blob/main/Projeto_Teste_Massivo_RGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando Bibliotecas:

In [42]:
'''importando a biblioteca Pandas -> usada para análise e manipulação de dados.'''
import pandas as pd

'''importando a biblioteca NumPy -> usada para trabalhar com matrizes e operações matemáticas.'''
import numpy as np

'''importando a biblioteca Matplotlib -> usada para criar visualizações de dados.'''
import matplotlib.pyplot as plt

'''importando a biblioteca OpenCV, uma biblioteca de código aberto -> amplamente utilizada para processamento de imagens.'''
import cv2

'''importando a classe KMeans da biblioteca Scikit-Learn, que é usada para realizar a clusterização de dados. 
A clusterização é uma técnica de aprendizado não supervisionado que agrupa dados semelhantes em clusters.'''
from sklearn.cluster import KMeans

'''importando a função cv2_imshow do Google Colab, que é usada para exibir imagens dentro do ambiente de notebook.'''
from google.colab.patches import cv2_imshow

'''A biblioteca glob é utilizada para encontrar todos os caminhos que correspondem a um determinado padrão. 
No contexto deste código, provavelmente será utilizada para encontrar imagens em um diretório.'''
import glob

import warnings

In [10]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


Essas duas linhas de código são específicas para o ambiente do Google Colab e são usadas para montar o Google Drive na sessão do Colab.
  * A primeira linha importa a biblioteca drive do Google Colab, que contém a função mount, que é utilizada para montar o Google Drive na sessão do Colab. 
  * A segunda linha chama a função mount e monta o Google Drive em uma pasta específica, neste caso em /content/drive/MyDrive/.

É importante destacar que essas duas linhas podem ser ignoradas se você estiver executando o código em um ambiente diferente do Google Colab ou se não precisar acessar arquivos armazenados no Google Drive.

## Lendo Imagem:

In [44]:
# Variável path_list armazena uma lista de caminhos de arquivos que correspondem ao padrão especificado.
path_list = glob.glob('/content/drive/MyDrive/Pesquisa_Cochonilha/Base_Imagens/Editadas_Cortadas/*.jpg')

## Processamento em Massa:

In [45]:
def extract_image(thresh, cnt):
  # Cria uma imagem em branco com as mesmas dimensões da imagem original
  im_blank = np.zeros(im.shape, np.uint8) 

  # Desenha o contorno especificado na imagem em branco
  cv2.drawContours(im_blank, [cnt], -1, 255, -1)

  # Cria uma máscara que corresponde aos pixels dentro do contorno
  mask = im_blank == 255

  # Aplica a máscara na imagem original para obter apenas os pixels dentro do contorno
  im_filter = im[mask]

  # Retorna a nova imagem
  return im_filter

In [ ]:
# Loop que processa cada arquivo na lista de caminhos de arquivos.
for link in path_list:

  warnings.filterwarnings("ignore", message="X does not have valid feature names, but KMeans was fitted with feature names")

  # The function cv2.imread() is used to read an image.
  im = cv2.imread(link)

  # Separa o nome especifico da imagem.
  nome_completo = link.split('/')
  nome = nome_completo[7].split('.')

  # Imprime as fronteiras e identifica as imagens:
  print("================================================================================")
  print("================================================================================") 
  print(nome[0])

  # Instrução para descobrir o tamanho da dimensão da imagem (Linhas x Colunas)
  print('Dim:' + str(im.shape))

  # Cria uma cópia da imagem original.
  compara = im.copy()

  # separa uma imagem em seus canais de cor individuais (B, G, R)
  im_split_channels = cv2.split(im)

  # realiza a mesma operação de separação de canais de cor em BGR e atribui cada canal de cor a uma variável correspondente
  (b, g, r) = cv2.split(im)

  # Aplica uma limiarização com método de Otsu à imagem.
  ret, thresh = cv2.threshold(b, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  rows, columns = thresh.shape
  areaImagem = rows*columns

  # Inicializa a variável areaOcupada como zero
  areaOcupada = 0

  # Loop sobre a lista de contornos encontrados na imagem binarizada
  for qtd in range(len(contours)):
    # Calcula a área correspondente ao contorno atual e adiciona à variável areaOcupada
    atual = cv2.contourArea(contours[qtd])
    areaOcupada = areaOcupada + atual

  # Exibe algumas informações relevantes na tela
  print("areaImagem = ", areaImagem)  # Exibe a área total da imagem binarizada
  print('Foram encontrados ' + str(len(contours)) + ' objetos com seus contornos')  # Exibe a quantidade de objetos encontrados
  print('Area total ocupada: ' + str(areaOcupada))  # Exibe a área total ocupada pelos objetos
  print('Relação infestação/área total: ' + str((areaOcupada/(rows*columns))*100) + ' %')  # Exibe a relação entre a área ocupada pelos objetos e a área total da imagem binarizada

  # Desenha os contornos encontrados na imagem original
  cv2.drawContours(im, contours, -1, (255,0,0), 2)  # Desenha os contornos em cor magenta com largura de linha 2 pixels

  areas = [] # Armazena a área de cada objeto
  perimeters = [] # Armazena o perímetro de cada objeto
  centroids_x = [] # Armazena as coordenadas x do centróide de cada objeto
  centroids_y = [] # Armazena as coordenadas y do centróide de cada objeto
  aspect_ratio = [] # Armazena a relação entre largura e altura do retângulo de contorno de cada objeto
  extent = [] # Armazena a razão entre a área do contorno e a área do retângulo de contorno de cada objeto
  solidity = [] # Armazena a razão entre a área do contorno e a área do casco convexo de cada objeto
  equivalent_diameter = [] # Armazena o diâmetro equivalente (em pixels) do círculo com a mesma área de cada objeto

  # r_mean, r_min, r_max e r_std: armazenarão a média, o mínimo, o máximo e o desvio padrão dos valores do canal vermelho (R) de cada objeto encontrado na imagem.
  r_mean = []
  r_min = []
  r_max = []
  r_std = []

  # g_mean, g_min, g_max e g_std: armazenarão a média, o mínimo, o máximo e o desvio padrão dos valores do canal verde (G) de cada objeto encontrado na imagem.
  g_mean = []
  g_min = []
  g_max = []
  g_std = []

  # b_mean, b_min, b_max e b_std: armazenarão a média, o mínimo, o máximo e o desvio padrão dos valores do canal azul (B) de cada objeto encontrado na imagem.
  b_mean = []
  b_min = []
  b_max = []
  b_std = []

  width = [] # Armazena a largura do retângulo de contorno de cada objeto
  height = [] # Armazena a altura do retângulo de contorno de cada objeto
  angle = [] # Armazena o ângulo (em graus) do retângulo de contorno de cada objeto
  radius = [] # Armazena o raio do círculo de contorno de cada objeto

  # Para cada contorno da lista de contornos:
  for c in contours:

    # Momentos do objeto
    M = cv2.moments(c)

    # Área do objeto
    areas.append(M['m00'])

    # Perímetro do objeto
    perimeters.append(cv2.arcLength(c,True)) #if not convex, False

    # Proporção de aspecto do objeto
    x,y,w,h = cv2.boundingRect(c)
    aspect_ratio.append(float(w)/h)
    width.append(w)
    height.append(h)

    # Ângulo e raio mínimo envolvente em torno do objeto
    rect = cv2.minAreaRect(c)
    circle = cv2.minEnclosingCircle(c)
    angle.append(rect[2])
    radius.append(circle[1])

    # Extensão do objeto (razão entre a área do objeto e a área do retângulo envolvente mínimo)
    rect_area = w*h
    if rect_area > 0:
        extent.append(M['m00']/rect_area)
    else: 
        extent.append(0)

    # Solidez do objeto (razão entre a área do objeto e a área de seu casco convexo)
    hull = cv2.convexHull(c)
    hull_area = cv2.contourArea(hull)
    if hull_area > 0:
        solidity.append(M['m00']/hull_area)
    else:
        solidity.append(0)

    # Diâmetro equivalente do objeto (diâmetro do círculo cuja área é a mesma da área do objeto)
    equivalent_diameter = np.sqrt(4*M['m00']/np.pi)

    # Cálculo das coordenadas do centroide do objeto
    if M['m00'] != 0:
        centroids_x.append(M['m10']/M['m00'])
        centroids_y.append(M['m01']/M['m00'])
    else:
        centroids_x.append(0)
        centroids_y.append(0)
        
    # Extração das informações de cor do objeto nas bandas R, G e B
    r = extract_image(im_split_channels[2], c)
    g = extract_image(im_split_channels[1], c)
    b = extract_image(im_split_channels[0], c)

    # Cálculo das estatísticas de cor para as bandas R, G e B
    # Canal vermelho:
    r_mean.append(np.mean(r))
    r_max.append(np.max(r))
    r_min.append(np.min(r))
    r_std.append(np.std(r))

    # Canal verde:
    g_mean.append(np.mean(g))
    g_max.append(np.max(g))
    g_min.append(np.min(g))
    g_std.append(np.std(g))

    # Canal Azul:
    b_mean.append(np.mean(b))
    b_max.append(np.max(b))
    b_min.append(np.min(b))
    b_std.append(np.std(b))

  features = {'r_mean': r_mean, 'r_max': r_max, 'r_min': r_min, 'r_std': r_std,
            'g_mean': g_mean, 'g_max': g_max, 'g_min': g_min, 'g_std': g_std,
            'b_mean': b_mean, 'b_max': b_max, 'b_min': b_min, 'b_std': b_std,
            'area': areas, 'perimiter': perimeters, 'aspect_ratio': aspect_ratio, 
            'extent': extent, 'solidity': solidity, 'equivalent_diameter': equivalent_diameter
  } 

  df = pd.DataFrame(features)

  # n_clusters: Define o número de clusters:
  n_clusters = 3

  if len(df) >= n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10, algorithm='full')
    kmeans.fit(df)
  else:
    print("Número de amostras é menor que o número de clusters desejados.")

  im2 = compara.copy()

  # Lista vazia conts para armazenar os contornos agrupados por classes
  conts = [[],[],[],[]] #contour list

  # Loop através dos contornos e prevê sua classe utilizando KMeans.predict
  for cont, cnt in enumerate(contours):
    # Adiciona o contorno na lista correspondente à sua classe
    class_ = kmeans.predict([df.iloc[cont]])[0]
    conts[class_].append(cnt)

  # Desenha os contornos agrupados por classe na imagem   
  if n_clusters == 3:
    im2 = cv2.drawContours(im2, conts[0], -1, (255, 0, 0), thickness=2) # Azul
    im2 = cv2.drawContours(im2, conts[1], -1, (0, 255, 0), thickness=2) # Laranja
    im2 = cv2.drawContours(im2, conts[2], -1, (0, 0, 255), thickness=2) # Verde
  elif n_clusters == 2:
    im2 = cv2.drawContours(im2, conts[0], -1, (255, 255, 0), thickness=2)
    im2 = cv2.drawContours(im2, conts[1], -1, (0, 255, 255), thickness=2)

  print("O numero de cluster escolhido foi: " + str(n_clusters))
  print()
  print('class 0: ' + str(len(conts[0])) + ' individuals')
  print('class 1: ' + str(len(conts[1])) + ' individuals')
  print('class 2: ' + str(len(conts[2])) + ' individuals')
  print('class 3: ' + str(len(conts[3])) + ' individuals')

  cv2_imshow(cv2.hconcat([compara, im, im2]))